In [6]:
import pandas as pd

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
pd.options.display.float_format = '{:.2f}'.format

# 기관별 정원, 인구, 사업체, 종사자

In [9]:
df_org = pd.read_csv("./(250617)근로감독관 충원 RAW DATA(v0.2).csv", 
                     usecols = ['기관명', '과수(전체)','전체정원','감독관','인구수','사업체수','종사자수','신고사건접수건수'], 
                     encoding = 'cp949')

In [10]:
df_org.head()

,기관명,과수(전체),전체정원,감독관,인구수,사업체수,종사자수,신고사건접수건수
0,서울청,17,297,146,1007159,96342,1118883,11789
1,서울강남지청,8,161,88,556506,60696,730965,12226
2,서울동부지청,8,187,82,1750181,73619,735728,10836
3,서울서부지청,7,176,71,1323643,52899,528901,8233
4,서울남부지청,8,180,75,1353895,60319,719842,9804


# 감독관 100명 증원

##### 프로세스 설명 : 1.감독관 1인당 담당량을 계산  2. 1인당 담당량이 가장 만은 지청에 감독관 1명을 증원  3.  1,2 반복


In [11]:
df_증원 = df_org.copy()
df_증원['반복회수'] = 0
df_증원 = df_증원[['반복회수'] + [col for col in df_증원.columns if col != '반복회수']]
plus_arr = []
df_증원log = pd.DataFrame(columns = df_증원.columns)
df_증원기관 = pd.DataFrame(columns = ['반복회차', '기관명', '현감독관', '작업후_감독관'])

In [12]:
for i in range(0,100):

    ## 변수별 감독관 1인당 담당량 계산
    df_증원['1인당_인구수'] = df_증원['인구수']/df_증원['감독관']
    df_증원['1인당_사업체수'] = df_증원['사업체수']/df_증원['감독관']
    df_증원['1인당_종사자수'] = df_증원['종사자수']/df_증원['감독관']
    df_증원['1인당_신고사건접수건수'] = df_증원['신고사건접수건수']/df_증원['감독관']

    ### 1인당 총 업무량(Point) 계산
    df_증원['1인당_업무량'] = df_증원['1인당_인구수'] + df_증원['1인당_사업체수'] + df_증원['1인당_종사자수'] + df_증원['1인당_신고사건접수건수']

    ### 1인당 업무량 높은 순 정렬
    df_증원 = df_증원.sort_values(['1인당_업무량'], ascending = False).copy().reset_index(drop=True)
    df_증원['반복회수'] = i
    
    #로그 저장
    df_증원log = pd.concat([df_증원log, df_증원],axis=0)
    tmp = pd.DataFrame({'반복회차' : [i], 
                        '기관명':[df_증원.loc[0, '기관명']], 
                        '현감독관':[df_org[df_org['기관명'] == df_증원.loc[0, '기관명']].reset_index().loc[0, '감독관']],
                        '작업후_감독관': [df_증원.loc[0, '감독관']]})
    df_증원기관 = pd.concat([df_증원기관, tmp], axis=0)


    ### 상위 1번 지청에 감독관 1명 증원
    df_증원.loc[0, '감독관'] += 1
    plus_arr.append(df_증원.loc[0, '기관명'])


In [13]:
df_증원log = df_증원log[df_증원.columns]

In [14]:
df_증원log.to_excel("반복회차별 데이터.xlsx")
df_증원기관.to_excel("반복회차별 증원된 기관.xlsx")

In [15]:
#with pd.ExcelWriter("결과물2.xlsx", engine='openpyxl') as writer:
#    df_증원log.to_excel(writer, sheet_name = "반복회차별 데이터", index=False)
#    df_증원기관.to_excel(writer, sheet_name = "반복회차별 증원된 기관", index=False)